In [1]:
import numpy as np
import pandas as pd


In [11]:


# example
data = {
    'HEATING_COST_CURRENT': [954],
    'TOTAL_FLOOR_AREA': [100.0],
    'ENERGY_CONSUMPTION_CURRENT': [181],
    'MAIN_FUEL': ['mains gas'],
    'LIGHTING_COST_CURRENT': [127],
    'HOT_WATER_COST_CURRENT': [213]
}

df = pd.DataFrame(data)

# Constants based on the provided table and SAP methodology
fuel_data = {
    "mains gas": {"price_per_kwh": 0.0348, "primary_energy_factor": 1.22, "co2_emission_factor": 0.216},
    "bulk LPG": {"price_per_kwh": 0.0760, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
    "bottled LPG": {"price_per_kwh": 0.1030, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
    # Add other fuel types as needed
}
boil_data = {
    "main gas": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.22, "co2_emission_factor": 0.216},
    "LPG": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
}
# Extract relevant fuel data based on MAIN_FUEL
df['Fuel_Data'] = df['MAIN_FUEL'].apply(lambda x: fuel_data.get(x.strip().lower(), fuel_data["mains gas"]))

# Calculate Heating Final Energy Intensity [kWh/m2/year]
df['Heating_Final_Energy_Intensity'] = df.apply(lambda row: row['HEATING_COST_CURRENT'] / (row['Fuel_Data']['price_per_kwh'] * row['TOTAL_FLOOR_AREA']), axis=1)

# Convert Heating Final Energy Intensity [kWh/m2/year] (final) into Heating Primary Energy Intensity [kWh/m2/year] (primary)
df['Heating_Primary_Energy_Intensity'] = df.apply(lambda row: row['Heating_Final_Energy_Intensity'] * row['Fuel_Data']['primary_energy_factor'], axis=1)

# Check if the sum on all usages (heating, lighting etc) of calculated Primary energy intensities [kWh/m2/year] (primary) equals ENERGY_CONSUMPTION_CURRENT
Lighting_Primary_Factor = 3.07
HotWater_Primary_Factor = 1.22

df['Lighting_Final_Energy_Intensity'] = df.apply(lambda row: row['LIGHTING_COST_CURRENT'] / (0.1319 * row['TOTAL_FLOOR_AREA']), axis=1)
df['HotWater_Primary_Energy_Intensity'] = df.apply(lambda row: row['HOT_WATER_COST_CURRENT'] / (0.0424 * row['TOTAL_FLOOR_AREA']), axis=1)

df['Total_Primary_Energy_Intensity'] = df.apply(lambda row: row['Heating_Primary_Energy_Intensity'] + (row['Lighting_Final_Energy_Intensity']*Lighting_Primary_Factor) 
                                                + (row['HotWater_Primary_Energy_Intensity'] * HotWater_Primary_Factor), axis=1)

df['Total_Primary_Energy_Consumption'] = df['Total_Primary_Energy_Intensity'] * df['TOTAL_FLOOR_AREA']
df['Energy_Consumption_Check'] = df['Total_Primary_Energy_Consumption'] == df['ENERGY_CONSUMPTION_CURRENT']

# Calculate CO2 EMISS CURR PER FLOOR AREA [kgCO2eq/m2/year] from Final energy intensities and pertinent emission factors [kgCO2eq/kWh]
df['CO2_Emissions_per_Floor_Area'] = df.apply(lambda row: row['Heating_Final_Energy_Intensity'] * row['Fuel_Data']['co2_emission_factor'], axis=1)


# 显示计算结果
print(df[['Heating_Final_Energy_Intensity', 'Heating_Primary_Energy_Intensity', 'Total_Primary_Energy_Intensity', 'Energy_Consumption_Check', 'CO2_Emissions_per_Floor_Area']])


   Heating_Final_Energy_Intensity  Heating_Primary_Energy_Intensity  \
0                      274.137931                        334.448276   

   Total_Primary_Energy_Intensity  Energy_Consumption_Check  \
0                      425.295526                     False   

   CO2_Emissions_per_Floor_Area  
0                     59.213793  


In [ ]:
# Calculate the EPC scores CURRENT ENERGY EFFICIENCY and CURRENT ENERGY RATING
# SAP rating calculation (example formula, the actual may differ)
Energy_Cost_Deflator = 0.42
Current_Energy_Efficiency = 72

df['SAP_Rating'] = 

# Convert SAP Rating to EPC Rating
def calculate_epc_rating(sap_rating):
    if sap_rating >= 92:
        return 'A'
    elif sap_rating >= 81:
        return 'B'
    elif sap_rating >= 69:
        return 'C'
    elif sap_rating >= 55:
        return 'D'
    elif sap_rating >= 39:
        return 'E'
    elif sap_rating >= 21:
        return 'F'
    else:
        return 'G'

df['EPC_Rating'] = df['SAP_Rating'].apply(calculate_epc_rating)


In [2]:
file_path = '/Users/siruiwang/Downloads/epc-certificate-2/certificates.csv'
data_57 = pd.read_csv(file_path)
df = pd.DataFrame(data_57)

display(df.columns)


Index(['LMK_KEY', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE',
       'BUILDING_REFERENCE_NUMBER', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'POTENTIAL_ENERGY_EFFICIENCY', 'PROPERTY_TYPE', 'BUILT_FORM',
       'INSPECTION_DATE', 'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY',
       'LODGEMENT_DATE', 'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT',
       'ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_CURRENT',
       'ENERGY_CONSUMPTION_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISS_CURR_PER_FLOOR_AREA', 'CO2_EMISSIONS_POTENTIAL',
       'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
       'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL',
       'HOT_WATER_COST_CURRENT', 'HOT_WATER_COST_POTENTIAL',
       'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF', 'MAINS_GAS_FLAG', 'FLOOR_LEVEL',
       'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT', 'MAIN_HEATING_CONTROLS',
       'MULTI_GLAZE_PROPORTION', 'GLAZED_TYPE', 'GLAZED_AREA',
      

In [3]:
fuel_data = {
    "mains gas": {"price_per_kwh": 0.0348, "primary_energy_factor": 1.22, "co2_emission_factor": 0.216},
    "bulk LPG": {"price_per_kwh": 0.0760, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
    "bottled LPG": {"price_per_kwh": 0.1030, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
    "LPG subject to Special Condition 18": {"price_per_kwh": 0.0348, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
    "biogas (including anaerobic digestion)": {"price_per_kwh": 0.0760, "primary_energy_factor": 1.10, "co2_emission_factor": 0.098},
    "heating oil": {"price_per_kwh": 0.0544, "primary_energy_factor": 1.10, "co2_emission_factor": 0.298},
    "biodiesel from any biomass source": {"price_per_kwh": 0.0764, "primary_energy_factor": 1.06, "co2_emission_factor": 0.123},
    "biodiesel from vegetable oil only": {"price_per_kwh": 0.0764, "primary_energy_factor": 1.01, "co2_emission_factor": 0.083},
    "appliances able to use mineral oil or biodiesel": {"price_per_kwh": 0.0544, "primary_energy_factor": 1.10, "co2_emission_factor": 0.298},
    "B30K": {"price_per_kwh": 0.0610, "primary_energy_factor": 1.09, "co2_emission_factor": 0.245},
    "bioethanol from any biomass source": {"price_per_kwh": 0.0470, "primary_energy_factor": 1.08, "co2_emission_factor": 0.140},
    "house coal": {"price_per_kwh": 0.0367, "primary_energy_factor": 1.00, "co2_emission_factor": 0.394},
    "anthracite": {"price_per_kwh": 0.0364, "primary_energy_factor": 1.00, "co2_emission_factor": 0.394},
    "manufactured smokeless fuel": {"price_per_kwh": 0.0461, "primary_energy_factor": 1.21, "co2_emission_factor": 0.433},
    "wood logs": {"price_per_kwh": 0.0423, "primary_energy_factor": 1.04, "co2_emission_factor": 0.019},
    "wood pellets (in bags for secondary heating)": {"price_per_kwh": 0.0581, "primary_energy_factor": 1.26, "co2_emission_factor": 0.039},
    "wood pellets (bulk supply for main heating)": {"price_per_kwh": 0.0526, "primary_energy_factor": 1.26, "co2_emission_factor": 0.039},
    "wood chips": {"price_per_kwh": 0.0307, "primary_energy_factor": 1.12, "co2_emission_factor": 0.016},
    "dual fuel appliance (mineral and wood)": {"price_per_kwh": 0.0399, "primary_energy_factor": 1.02, "co2_emission_factor": 0.226},
    "standard tariff": {"price_per_kwh": 0.1319, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "7-hour tariff (high rate)": {"price_per_kwh": 0.1529, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "7-hour tariff (low rate)": {"price_per_kwh": 0.0550, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "10-hour tariff (high rate)": {"price_per_kwh": 0.1468, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "10-hour tariff (low rate)": {"price_per_kwh": 0.0750, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "18-hour tariff (high rate)": {"price_per_kwh": 0.1367, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "18-hour tariff (low rate)": {"price_per_kwh": 0.0741, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "24-hour heating tariff": {"price_per_kwh": 0.0661, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "electricity sold to grid": {"price_per_kwh": 0.1319, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "electricity displaced from grid": {"price_per_kwh": 0.0519, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "electricity, any tariff": {"price_per_kwh": 0.1319, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "heat from boilers - mains gas": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.22, "co2_emission_factor": 0.216},
    "heat from boilers - LPG": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.09, "co2_emission_factor": 0.241},
    "heat from boilers - oil": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.10, "co2_emission_factor": 0.331},
    "heat from boilers that can use mineral oil or biodiesel": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.10, "co2_emission_factor": 0.331},
    "heat from boilers using biodiesel from any biomass source": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.06, "co2_emission_factor": 0.123},
    "heat from boilers using biodiesel from vegetable oil only": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.01, "co2_emission_factor": 0.083},
    "heat from boilers - B30K": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.09, "co2_emission_factor": 0.269},
    "heat from boilers - coal": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.00, "co2_emission_factor": 0.380},
    "heat from electric heat pump": {"price_per_kwh": 0.0424, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "heat from boilers - waste combustion": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.23, "co2_emission_factor": 0.047},
    "heat from boilers - biomass": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.01, "co2_emission_factor": 0.031},
    "heat from boilers - biogas (landfill or sewage gas)": {"price_per_kwh": 0.0424, "primary_energy_factor": 1.10, "co2_emission_factor": 0.098},
    "waste heat from power station": {"price_per_kwh": 0.0297, "primary_energy_factor": 1.34, "co2_emission_factor": 0.058},
    "geothermal heat source": {"price_per_kwh": 0.0297, "primary_energy_factor": 1.24, "co2_emission_factor": 0.041},
    "heat from CHP": {"price_per_kwh": 0.0297, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "electricity generated by CHP": {"price_per_kwh": 0.0519, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519},
    "electricity for pumping in distribution network": {"price_per_kwh": np.nan, "primary_energy_factor": 3.07, "co2_emission_factor": 0.519}
}


In [22]:
# Calculate Heating Final Energy Intensity [kWh/m2/year]

def get_fuel_data(fuel_type):
    fuel_type = fuel_type.strip().lower()
    if fuel_type in fuel_data:
        return fuel_data[fuel_type]
    else:
        return fuel_data["mains gas"]

df['Fuel_Data'] = df['MAIN_FUEL'].apply(get_fuel_data)

print(df['Fuel_Data'])

def get_hot_water_fuel_data(row):
    description = row['HOTWATER_DESCRIPTION'].lower()
    main_heat_description = row['MAINHEAT_DESCRIPTION'].lower()

    if 'from main system' in description:
        if 'boiler and radiators, mains gas' in main_heat_description:
            return fuel_data['heat from boilers - mains gas']
        # 添加其他匹配逻辑
        # if 'boiler, oil' in main_heat_description:
        #     return fuel_data['boiler from oil']
        # if 'electric' in main_heat_description:
        #     return fuel_data['electric boiler']
    # 如果没有匹配的描述，返回默认的 mains gas
    return fuel_data['heat from boilers - mains gas']


df['Fuel_Data'] = df['MAIN_FUEL'].apply(get_fuel_data)
df['Hot_Water_Fuel_Data'] = df.apply(get_hot_water_fuel_data, axis=1)
df['Lighting_Fuel_Data'] = df.apply(lambda _: fuel_data['standard tariff'], axis = 1)

df['Heating_Final_Energy_Intensity'] = df.apply(lambda row: row['HEATING_COST_CURRENT'] / (row['Fuel_Data']['price_per_kwh'] * row['TOTAL_FLOOR_AREA']), axis=1)
df['Hot_Water_Final_Energy_Intensity'] = df.apply(lambda row: row['HOT_WATER_COST_CURRENT'] / (row['Hot_Water_Fuel_Data']['price_per_kwh'] * row['TOTAL_FLOOR_AREA']), axis=1)
df['Lighting_Final_Energy_Intensity'] = df.apply(lambda row: row['LIGHTING_COST_CURRENT'] / (row['Lighting_Fuel_Data']['price_per_kwh'] * row['TOTAL_FLOOR_AREA']), axis=1)


df[['Heating_Final_Energy_Intensity', 'Hot_Water_Final_Energy_Intensity', 'Lighting_Final_Energy_Intensity']]

0    {'price_per_kwh': 0.0348, 'primary_energy_fact...
Name: Fuel_Data, dtype: object


,Heating_Final_Energy_Intensity,Hot_Water_Final_Energy_Intensity,Lighting_Final_Energy_Intensity
0,315.407772,67.94699,12.004043


In [23]:
# Convert Heating Final Energy Intensity [kWh/m2/year] (final) into Heating Primary Energy Intensity [kWh/m2/year] (primary)

df['Heating_Primary_Energy_Intensity'] = df.apply(lambda row: row['Heating_Final_Energy_Intensity'] * row['Fuel_Data']['primary_energy_factor'], axis=1)
df['Hot_Water_Primary_Energy_Intensity'] = df.apply(lambda row: row['Hot_Water_Final_Energy_Intensity'] * row['Hot_Water_Fuel_Data']['primary_energy_factor'], axis=1)
df['Lighting_Primary_Energy_Intensity'] = df.apply(lambda row: row['Lighting_Final_Energy_Intensity'] * row['Lighting_Fuel_Data']['primary_energy_factor'], axis=1)

df[['Heating_Primary_Energy_Intensity', 'Hot_Water_Primary_Energy_Intensity', 'Lighting_Primary_Energy_Intensity']]

,Heating_Primary_Energy_Intensity,Hot_Water_Primary_Energy_Intensity,Lighting_Primary_Energy_Intensity
0,384.797482,82.895328,36.852413


In [28]:
# Check if the sum on all usages (heating, lighting etc) of calculated Primary energy intensities [kWh/m2/year] (primary) equals ENERGY_CONSUMPTION_CURRENT

df['all_usage'] = (df['Heating_Primary_Energy_Intensity'] + df['Lighting_Primary_Energy_Intensity'] + df['Hot_Water_Primary_Energy_Intensity']) 

df['Energy_Consumption_Check'] = df['all_usage'] == df['ENERGY_CONSUMPTION_CURRENT']

df[['all_usage', 'Energy_Consumption_Check']]

,all_usage,Energy_Consumption_Check
0,504.545224,False


In [17]:
# Calculate CO2 EMISS CURR PER FLOOR AREA [kgCO2eq/m2/year] from Final energy intensities and pertinent emission factors [kgCO2eq/kWh]
df['Heating_CO2_Emissions_per_Floor_Area'] = df.apply(lambda row: row['Heating_Final_Energy_Intensity'] * row['Fuel_Data']['co2_emission_factor'], axis=1)
df['Hot_Water_CO2_Emissions_per_Floor_Area'] = df.apply(lambda row: row['Hot_Water_Final_Energy_Intensity'] * row['Hot_Water_Fuel_Data']['co2_emission_factor'], axis=1)
df['Lighting_CO2_Emissions_per_Floor_Area'] = df.apply(lambda row: row['Lighting_Final_Energy_Intensity'] * row['Lighting_Fuel_Data']['co2_emission_factor'], axis=1)

df['Total_CO2_Emissions_per_Floor_Area'] = df['Heating_CO2_Emissions_per_Floor_Area'] + df['Hot_Water_CO2_Emissions_per_Floor_Area'] + df['Lighting_CO2_Emissions_per_Floor_Area']

df[['Total_CO2_Emissions_per_Floor_Area']]

,Total_CO2_Emissions_per_Floor_Area
0,89.034727


In [30]:
# Calculate the EPC scores CURRENT ENERGY EFFICIENCY and CURRENT ENERGY RATING
# SAP rating calculation (example formula, the actual may differ)

df['ECF'] = (df['HEATING_COST_CURRENT'] + df['HOT_WATER_COST_CURRENT'] + df['LIGHTING_COST_CURRENT']) * 0.42 / (df['TOTAL_FLOOR_AREA'] + 45)

df[['ECF']]

def SAP(ecf):
    
    if ecf >= 3.5:
        return 117-121*np.log10(ecf)
    else:
        return 100-13.95*ecf
    

df['SAP_factor'] = df['ECF'].apply(SAP)

df['SAP_factor'] = df['SAP_factor'].fillna(0)

def calculate_epc_rating(sap_rating):
    if sap_rating >= 92:
        return 'A'
    elif sap_rating >= 81:
        return 'B'
    elif sap_rating >= 69:
        return 'C'
    elif sap_rating >= 55:
        return 'D'
    elif sap_rating >= 39:
        return 'E'
    elif sap_rating >= 21:
        return 'F'
    else:
        return 'G'

df['EPC_Rating'] = df['SAP_factor'].apply(calculate_epc_rating)

df[['ECF', 'SAP_factor', 'EPC_Rating']]

,ECF,SAP_factor,EPC_Rating
0,4.222791,41.302454,E
